# Basic NLP hans-on

##### 使用するモジュールをインポートする

In [1]:
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from janome.tokenizer import Tokenizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

##### 読み込むファイルのパスを設定
[word2vec学習済みモデル](http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/)
> 解凍したファイル内の`entity_vector/entity_vector.model.bin`を設定

In [2]:
w2v_path = './data/entity_vector.model.bin'

## 自然言語のベクトル化

word2vecのモデルを読み込む(`%%time`は追加して実行時間を計測するため)

In [3]:
%%time
w2v_model = KeyedVectors.load_word2vec_format(w2v_path, binary=True)

CPU times: user 16.5 s, sys: 13.8 s, total: 30.3 s
Wall time: 30.5 s


#### 単語のベクトル化

読み込んだモデルを使って`猫`という単語のベクトルを取得してみる  
ベクトル自身と次元数を確認する

In [4]:
cat_vec = w2v_model['猫']
print(cat_vec)
print(cat_vec.shape)

[-0.43024188 -0.04794875 -1.2166483  -0.00707717  1.0666201   2.0926762
  1.2363118   0.09632737 -0.8704978   0.13878527 -0.2398951  -1.4510432
 -1.035872   -0.36443335  1.510419   -0.8976701  -0.43222582 -3.044292
 -0.4697854   0.8835323  -0.76157856 -1.5505594  -0.3028883  -0.89064634
  0.03595268  0.7125332   2.511127   -2.8003094  -0.9659015   1.0950842
  0.6309901   1.4498174  -0.66543955  2.7736151   0.73611355 -0.8395903
 -2.5741649   1.4304208   0.2820381   0.5554876   1.3982109  -0.48366612
 -0.89337915 -0.67719775 -0.17423566  0.30680582 -1.2953538   0.04251587
  0.85199183 -0.48973852 -0.86662847 -1.2539532   2.46837     2.9143462
  1.6813743  -1.9618986   0.36637524  1.2169962   0.61634284  0.7284531
  0.11117083 -1.0564184  -1.043863   -0.9271703  -0.86626095  0.5458255
 -2.590363    2.1277132  -0.04561193  1.2624754  -2.2917085   0.47231165
 -0.76928276  1.4070219   0.52328646 -0.12818593  1.8500334   2.3609638
 -0.87362415  0.08904377 -1.1197895  -0.17823507 -0.17032406 

`猫`と似たベクトルを持つ単語を10件取得する

In [5]:
w2v_model.most_similar(positive=[cat_vec], negative=[], topn=10)
# ここで[]がついているのはwikipediaのリンクが作成されていたもの

[('猫', 0.9999999403953552),
 ('[ネコ]', 0.845432698726654),
 ('ネコ', 0.8190228939056396),
 ('[猫]', 0.8119831681251526),
 ('ウサギ', 0.8038156032562256),
 ('小鳥', 0.7744659781455994),
 ('子猫', 0.7610605955123901),
 ('[犬]', 0.7583129405975342),
 ('ネズミ', 0.7523773908615112),
 ('子犬', 0.7486449480056763)]

##### ベクトルの計算をしてみる
日本 - 東京 + ニューヨーク = ???

In [6]:
w2v_model.most_similar(positive=['ニューヨーク', '日本'], negative=['東京'], topn=10)

[('アメリカ', 0.7917619943618774),
 ('アメリカ合衆国', 0.7624144554138184),
 ('[アメリカ合衆国]', 0.7213078737258911),
 ('[アメリカ]', 0.7131723165512085),
 ('イギリス', 0.7096747159957886),
 ('米国', 0.6873115301132202),
 ('[イギリス]', 0.6620149612426758),
 ('英国', 0.6602476835250854),
 ('ヨーロッパ', 0.6132903695106506),
 ('[ジャマイカ]', 0.6010661721229553)]

単語を入力するとベクトルを返してくれる関数を作成する

In [7]:
def get_wv(w):
    try:
        return w2v_model[w]
    # 未知語はkeyErrorとなってしまう
    except KeyError:
        return np.zeros(200)

#### 文章のベクトル化

形態素解析ツール`janome`を使ってみる  
まずは「私はサッカーが好きです。」という文章を形態素解析してみる

In [8]:
tokenizer = Tokenizer()
tokens = tokenizer.tokenize('私はサッカーが好きです。')
for token in tokens:
    print(token)

私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
サッカー	名詞,一般,*,*,*,*,サッカー,サッカー,サッカー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
好き	名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。


同じ文章の分ち書きを取得する

In [9]:
sep_text = [token for token in tokenizer.tokenize('私はサッカーが好きです。', wakati=True)]
print(sep_text)

['私', 'は', 'サッカー', 'が', '好き', 'です', '。']


同じ文章の文章ベクトルを取得する

In [10]:
doc_vec = sum([get_wv(w) for w in sep_text])/len(sep_text)
print(doc_vec)
print(doc_vec.shape)

[ 0.6273657  -0.5029469  -0.43460184 -1.1501808   0.12501477  0.73670965
  0.7365438   0.7396611  -0.8707982  -0.71904796 -0.40994534  0.25132483
 -2.3795378   0.18605451  0.9973677   0.8165956   0.74830896  0.7912072
  0.8549124  -0.09605421 -0.5207016   0.7937451   0.37317356  0.7891353
  1.1250448   1.5913105   0.2653513  -0.18871689 -1.245264    1.6019698
 -0.40631527  0.82664573 -0.59559447  1.7971992   0.3909776  -0.9302103
 -2.1869888   0.9312015   1.7912958  -0.38414487  1.2875404  -1.0330919
 -0.887987   -0.36752728  0.64559895  1.8445798   0.58402777 -0.1593472
 -0.5781916   0.7868512  -0.19558856 -1.0682898   0.74934965  0.65315807
  0.16822092 -0.73000354  0.58957946  1.3434268  -0.46643257  0.9142896
 -0.1199066   1.1502657   0.95243496 -0.54707146 -1.4049109  -0.92839813
 -2.2109168   1.547659    0.2113847   0.19778265 -0.36238125 -1.6211599
 -0.9794191   0.02179667  0.47072715 -0.89195824  0.5193229   0.44201428
 -0.0883886  -0.7677713  -0.5235376   0.02272597 -1.4935454

文章を入力するとベクトルを返してくれる関数を作成する

In [11]:
def get_dv(text):
    sep_text = [token for token in tokenizer.tokenize(text, wakati=True)]
    return sum([get_wv(w) for w in sep_text])/len(sep_text)

##### 「私はサッカーが好きです。」に対する下記3文章の類似度を比べてみる
1. 私は野球が好きです。
2. 私はラーメンが好きです。
3. 今日は良い天気です。

各文章のベクトルを取得する

In [12]:
vec = get_dv('私はサッカーが好きです。')

vec1 = get_dv('私は野球が好きです。')
vec2 = get_dv('私はラーメンが好きです。')
vec3 = get_dv('今日はいい天気です。')

cos類似度を使って各文章間の距離を測ってみる

In [13]:
sim_vs1 = cosine_similarity([vec], [vec1])
sim_vs2 = cosine_similarity([vec], [vec2])
sim_vs3 = cosine_similarity([vec], [vec3])
print(sim_vs1)
print(sim_vs2)
print(sim_vs3)

[[0.9865114]]
[[0.966223]]
[[0.81469536]]


##### 上記ベクトル比較をまとめて行う

ベクトル結合のために次元数を増やす

In [14]:
vec1_ = vec1.reshape(1, 200)
vec2_ = vec2.reshape(1, 200)
vec3_ = vec3.reshape(1, 200)
print('org', vec1.shape)
print('new', vec1_.shape)

org (200,)
new (1, 200)


3つのベクトルを結合する

In [15]:
vecs = np.concatenate((vec1_, vec2_, vec3_), axis=0)
print(vecs.shape)

(3, 200)


3つの文章との距離をまとめて取得する

In [16]:
sim = cosine_similarity([vec], vecs)
print(sim)

[[0.9865114  0.96622294 0.81469536]]


## FAQ検索

検索対象のFAQのパスを設定

In [17]:
qa_path = './data/muscle_qa.csv'

##### FAQデータを読み込む
先頭3行を確認する

In [18]:
faq_df = pd.read_csv(qa_path)
faq_df.head(3)

,q_id,q_txt,a_txt
0,Q0000,○○って何？,トレーニング用語の場合は、まずはウェイトトレーニング用語辞典を見てみてください。
1,Q0001,これから筋トレ初めて夏にまにあいますか？,・現在9月の場合：来年の夏まで1年弱。頑張れば多少は見栄えがするようになるかもしれません。\...
2,Q0002,三ヶ月でムキムキマッチョになれますか？,無理です。最低3年は頑張って下さい。


検索対象をFAQの質問文とし、FAQ質問文のベクトルのリストを取得する

In [19]:
faq_vecs = np.array([get_dv(doc) for doc in faq_df.q_txt])
faq_vecs.shape

(36, 200)

文章を入力すると、各FAQとの類似度を計算してくれる関数を作る

In [20]:
def get_faq_sim(text):
    vec = get_dv(text)
    sim = cosine_similarity([vec], faq_vecs)
    return sim[0]

In [21]:
sim = get_faq_sim('睡眠時間は何時間くらいがよい？')
sim

array([0.51239399, 0.65547589, 0.70830506, 0.66406682, 0.77426082,
       0.68723169, 0.81671257, 0.76775282, 0.81081473, 0.71943028,
       0.782112  , 0.74304168, 0.75255838, 0.64864124, 0.73604025,
       0.76398584, 0.81590054, 0.81554473, 0.77035144, 0.65050797,
       0.69872552, 0.7581692 , 0.7501388 , 0.85945486, 0.75614279,
       0.69778384, 0.84415088, 0.78898606, 0.831815  , 0.81245325,
       0.80473677, 0.80900392, 0.85549673, 0.85143797, 0.81659689,
       0.78287876])

類似度の高い順にFAQのインデックスを返す関数を作成

In [22]:
sort_i = np.argsort(sim)[::-1]
sort_i

array([23, 32, 33, 26, 28,  6, 34, 16, 17, 29,  8, 31, 30, 27, 35, 10,  4,
       18,  7, 15, 21, 24, 12, 22, 11, 14,  9,  2, 20, 25,  5,  3,  1, 19,
       13,  0])

類似する上位5件のFAQを表示する

In [23]:
faq_df.loc[sort_i[:5]]

,q_id,q_txt,a_txt
23,Q0023,一日にどれぐらいのカロリーが必要ですか？,成人男子であれば約2500kcalぐらい必要です。筋トレ1時間でだいたい500kcalぐらい...
32,Q0032,何時間ぐらい寝たらいいのですか？,一日7時間ぐらいが理想です。短すぎると回復しません。8時間以上の睡眠は体液が濃くなって体に悪...
33,Q0033,休みの時は全く何もしない方が良いのですか？,少しぐらいの有酸素運動なら疲れを取ってくれます。
26,Q0026,一日に必要なタンパク質はどれぐらいですか？,トレーニングをしている人であれば除脂肪体重の2/1000から3/1000ぐらい必要です。仮に...
28,Q0028,プロテインの一回の量はどれぐらい取ればいい？,一般的な人が消化吸収できるプロテイン量が30gと言われているので、一回30gが基本です。個人...


# 教師あり学習

[KNB(京都コーパス)](http://nlp.ist.i.kyoto-u.ac.jp/kuntt/)から抽出されたデータのパスを設定

In [24]:
knb_path = './data/knb_data.csv'

##### KNBデータを読み込む
先頭3行を確認する

In [25]:
knb_df = pd.read_csv(knb_path)
knb_df.head(3)

,label,text
0,Gourmet,［グルメ］烏丸六角のおかき屋さん
1,Gourmet,六角堂の前にある、蕪村庵というお店に行ってきた。
2,Gourmet,おかきやせんべいの店なのだが、これがオイシイ。


文章をベクトルに変換する

In [26]:
x_train = np.array([get_dv(doc) for doc in knb_df.text])
x_train.shape

(1410, 200)

ラベルをIDに変換する  
Gourmetを0, Sportを1とする

In [27]:
dic = {'Gourmet':0, 'Sports':1}
y_train = np.array([dic[lab] for lab in knb_df.label])
y_train.shape

(1410,)

5Foldでデータ分割して学習する

In [28]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

各foldでのスコア(accuracy)を出して、リストに格納する

In [29]:
acc_lst = []
for train_idx, valid_idx in cv.split(x_train, y_train):
    trn_x = x_train[train_idx]
    val_x = x_train[valid_idx]

    trn_y = y_train[train_idx.tolist()]
    val_y = y_train[valid_idx.tolist()]
    
    clf = LogisticRegression()
    clf.fit(trn_x, trn_y)
    
    pred = clf.predict(val_x)
    acc = accuracy_score(pred, val_y)
    acc_lst.append(acc)

格納したスコアリストから平均スコアを導出する

In [30]:
sum(acc_lst)/len(acc_lst)

0.8631292360843371